In [4]:
# !pip install stop_words
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from stop_words import get_stop_words
import os
from datetime import datetime
import pickle

In [8]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial

# /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/ mypath

DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/official.csv" #replace with your data path
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [9]:
data = pd.read_csv(DATA_PATH)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,150) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Recoding Y as a binary variable with mobile money = 1 and the rest = 0

In [10]:
dataX = data.drop(['trx_mode_desc'], axis = 1)

In [11]:
X = dataX.loc[:,:] 

In [12]:
Y = data.trx_mode_desc

In [13]:
Y1 = []
for y in Y:
    if y == 'mobile money':
        Y1.append(1)
    else:
        Y1.append(0)
Y = Y1

In [33]:
for column in X.columns:
  if X[column].dtype == "object":
    X[column] = X[column].astype("|S")

In [34]:
dropCols = []
for column in X.columns:
  if X[column].nunique() > 150 or X[column].dtype == 'float64':
    dropCols.append(column)
X = X.drop(dropCols, axis=1)

In [ ]:
####################################### LOADING X
with open("Xpkl", 'rb') as ifile:
  X = pickle.load(ifile)

In [ ]:
############################### Program is crashing here itself due to lack of ram
xdf = pd.DataFrame(X.toarray())
#xdf.to_csv("X.csv")

In [ ]:
xdf.columns = ohc.get_feature_names_out()

In [ ]:
############################## Adding numeric columns back to X
for col in dropCols:
  xdf[col] = dataX[col]

In [ ]:
DC = ["first_trx_date_acc", "last_trx_date_acc", "trx_date"]
first_trx_date_acc_f = []
last_trx_date_acc_f = []
trx_date_f = []

i = 0
for dc in DC:
    
    L = []
        
    for d in xdf[dc]:
        try:
            dat = datetime.strptime(d.decode('UTF-8'), "%d%b%Y")
            f = dat.timestamp()
            L.append(f)
        except:
            L.append(np.nan)
    
    if i == 0:
        first_trx_date_acc_f = L
    if i == 1:
        last_trx_date_acc_f = L
    if i == 2:
        trx_date_f = L
        
    i += 1

In [ ]:
xdf['first_trx_date_acc_f'] = first_trx_date_acc_f
xdf["last_trx_date_acc_f"] = last_trx_date_acc_f
xdf["trx_date_f"] = trx_date_f

In [ ]:
Xd = xdf.drop(DC, axis = 1)

In [ ]:
dropCol2 = []
for col in Xd.columns:
    if Xd[col].dtype != "float64":
        dropCol2.append(col)
        print(col)

In [ ]:
Xd = Xd.drop(dropCol2, axis=1)

In [ ]:
Ximp = Xd.fillna(Xd.mean())

Starting with machine learning preprocessing using Ximp and Y

In [ ]:
# Import dependencies 
from sklearn.model_selection import train_test_split


In [ ]:
X = Xd
y = Y

Tree

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
XtrainVal, Xtest, YtrainVal, Ytest = train_test_split(X, Yd, test_size=0.2, random_state=42)
Xtrain, Xval, Ytrain, Yval = train_test_split(XtrainVal, YtrainVal, test_size=0.25, random_state=42)

In [ ]:
# Here we use the best model for the tree with a depth = 50, that had an MCC score of 92%
clf = DecisionTreeClassifier(random_state=42, max_depth=50)

In [ ]:
clf = clf.fit(Xtrain, Ytrain)

In [ ]:
YpredVal = clf.predict(Xval)
YpredTrain = clf.predict(Xtrain)

In [ ]:
# ROC AUC SCORE
